In [14]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### IOU

In [15]:
def calculate_iou(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    intersection = np.logical_and(true_mask_gray, predicted_mask_gray)
    union = np.logical_or(true_mask_gray, predicted_mask_gray)

    iou = np.sum(intersection) / np.sum(union)
    return iou

### Dice

In [16]:
def calculate_dice_score(true_mask, predicted_mask):
    # Ensure the masks are binary
    true_mask_binary = true_mask > 0
    predicted_mask_binary = predicted_mask > 0
    
    intersection = np.logical_and(true_mask_binary, predicted_mask_binary)

    dice_score = 0

    # Calculate Dice score only if both true and predicted masks are present
    if np.sum(true_mask_binary) > 0 and np.sum(predicted_mask_binary) > 0:
        # Calculate intersection area
        intersection_area = np.sum(intersection)
        # Calculate area of true mask
        true_area = np.sum(true_mask_binary)
        # Calculate area of predicted mask
        predicted_area = np.sum(predicted_mask_binary)
        
        # Calculate Dice score
        dice_score = 2 * intersection_area / (true_area + predicted_area)
    else:
        dice_score = 0

    return dice_score

### pixel wise accuracy

In [17]:
#pixel wise accuracy

def calculate_pixel_wise_accuracy(true_mask, predicted_mask):
    
    #convert true mask to gray scale
    true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
    #convert predicted mask to gray scale
    predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
    #calculate pixel wise accuracy
    #number of pixels in true mask
    true_positive=np.sum((true_mask_gray==255) & (predicted_mask_gray==255))
    false_positive=np.sum((true_mask_gray==0) & (predicted_mask_gray==255))
    false_negative=np.sum((true_mask_gray==255) & (predicted_mask_gray==0))
    true_negative=np.sum((true_mask_gray==0) & (predicted_mask_gray==0))


    #pixel wise accuracy
    pixel_wise_accuracy=(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)

    return pixel_wise_accuracy

In [18]:
#dataframe to store the results datset wise
results_df=pd.DataFrame(columns=['Dataset','IOU','Dice Score','Pixel Wise Accuracy'])

### Name of folder to eval

In [19]:
folder_name='SAM_predictions_BBox'

#delete benign (195)_mask_2_mask.png image from train/benign folder

benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)
print(list_of_benign_predictions_dir)
#delete benign (195)_mask_2_mask.png image from valid/benign folder
if 'benign (195)_mask_2_mask_prediction.png' in list_of_benign_predictions_dir:
    os.remove(os.path.join(benign_predictions_dir,'benign (195)_mask_2_mask_prediction.png'))

FileNotFoundError: [Errno 2] No such file or directory: '../SAM_predictions_BBox/train/benign'

In [20]:
def fix_sep(path):
    '''
    fixes the path separator to the current OS
    '''
    not_sep = {'\\': '/', '/': '\\'}

    if not_sep[os.sep] in path:
        return path.replace(not_sep[os.sep], os.sep)
    else:
        return path


In [21]:
def calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name):
    
    list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)

    df = pd.DataFrame(columns=['File', 'IOU', 'Dice Score', 'Pixel Wise Accuracy'])

    for files in list_of_benign_predictions_dir:
        # read true mask
        #print(files)
        true_mask = cv2.imread(os.path.join(benign_gt_dir, files.replace('_mask_prediction.png', '_mask.png')))
        # read predicted mask
        #print(files)
        predicted_mask = cv2.imread(os.path.join(benign_predictions_dir, files))
        
        

        # calculate iou
        iou = calculate_iou(true_mask, predicted_mask)
        # calculate dice score
        dice_score = calculate_dice_score(true_mask, predicted_mask)
        # calculate pixel wise accuracy
        pixel_wise_accuracy = calculate_pixel_wise_accuracy(true_mask, predicted_mask)

        df.loc[len(df)] = [files, iou, dice_score, pixel_wise_accuracy]


    # save the dataframe to a csv file
    df.to_csv(csv_file_name, index=False)
    
    print('Metrics saved to ',csv_file_name)
    average_dice_score=np.mean(df['Dice Score'])
    average_iou=np.mean(df['IOU'])
    average_pixel_wise_accuracy=np.mean(df['Pixel Wise Accuracy'])
    
    return average_dice_score,average_iou,average_pixel_wise_accuracy

## only box prompt

In [23]:
benign_predictions_dir=os.path.join('..', folder_name, 'valid' ,'benign')
benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
csv_file_name=os.path.join('..',folder_name,'valid','benign_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

benign (424)_mask_prediction.png
benign (424)_mask_prediction.png
benign (423)_mask_prediction.png
benign (423)_mask_prediction.png
benign (367)_mask_prediction.png
benign (367)_mask_prediction.png
benign (348)_mask_prediction.png
benign (348)_mask_prediction.png
benign (17)_mask_prediction.png
benign (17)_mask_prediction.png
benign (306)_mask_prediction.png
benign (306)_mask_prediction.png
benign (247)_mask_prediction.png
benign (247)_mask_prediction.png
benign (177)_mask_prediction.png
benign (177)_mask_prediction.png
benign (7)_mask_prediction.png
benign (7)_mask_prediction.png
benign (257)_mask_prediction.png
benign (257)_mask_prediction.png
benign (120)_mask_prediction.png
benign (120)_mask_prediction.png
benign (143)_mask_prediction.png
benign (143)_mask_prediction.png
benign (127)_mask_prediction.png
benign (127)_mask_prediction.png
benign (297)_mask_prediction.png
benign (297)_mask_prediction.png
benign (415)_mask_prediction.png
benign (415)_mask_prediction.png
benign (245)_mas

In [197]:
results_df

,Dataset,IOU,Dice Score,Pixel Wise Accuracy
0,../SAM_predictions_BBox/valid/benign_metrics.csv,0.302003,0.370326,0.935065


In [198]:
benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
csv_file_name=os.path.join('..',folder_name,'train','benign_metrics.csv')
average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score) 
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_BBox/train/benign_metrics.csv
Average Dice Score: 0.4076550237274527
Average IOU: 0.32458379342243304
Average Pixel Wise Accuracy: 0.92759963075654


In [199]:
results_df

,Dataset,IOU,Dice Score,Pixel Wise Accuracy
0,../SAM_predictions_BBox/valid/benign_metrics.csv,0.302003,0.370326,0.935065
1,../SAM_predictions_BBox/train/benign_metrics.csv,0.324584,0.407655,0.927600


In [22]:
malignant_predictions_dir=os.path.join('..', folder_name, 'valid' ,'malignant')
malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
csv_file_name=os.path.join('..',folder_name,'valid','malignant_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

malignant (106)_mask_prediction.png
malignant (106)_mask_prediction.png
malignant (104)_mask_prediction.png
malignant (104)_mask_prediction.png
malignant (121)_mask_prediction.png
malignant (121)_mask_prediction.png
malignant (199)_mask_prediction.png
malignant (199)_mask_prediction.png
malignant (49)_mask_prediction.png
malignant (49)_mask_prediction.png
malignant (28)_mask_prediction.png
malignant (28)_mask_prediction.png
malignant (184)_mask_prediction.png
malignant (184)_mask_prediction.png
malignant (107)_mask_prediction.png
malignant (107)_mask_prediction.png
malignant (185)_mask_prediction.png
malignant (185)_mask_prediction.png
malignant (168)_mask_prediction.png
malignant (168)_mask_prediction.png
malignant (136)_mask_prediction.png
malignant (136)_mask_prediction.png
malignant (175)_mask_prediction.png
malignant (175)_mask_prediction.png
malignant (61)_mask_prediction.png
malignant (61)_mask_prediction.png
malignant (171)_mask_prediction.png
malignant (171)_mask_prediction.pn

In [201]:
malignant_predictions_dir=os.path.join('..', folder_name, 'train' ,'malignant')
malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
csv_file_name=os.path.join('..',folder_name,'train','malignant_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_BBox/train/malignant_metrics.csv
Average Dice Score: 0.3929783705016516
Average IOU: 0.2812295604343984
Average Pixel Wise Accuracy: 0.8740815792831448


## with box and point prompt

In [29]:
folder_name='SAM_predictions_BBoxPoint'

#delete benign (195)_mask_2_mask.png image from train/benign folder

benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)
print(list_of_benign_predictions_dir)
#delete benign (195)_mask_2_mask.png image from valid/benign folder
if 'benign (195)_mask_2_mask_prediction.png' in list_of_benign_predictions_dir:
    os.remove(os.path.join(benign_predictions_dir,'benign (195)_mask_2_mask_prediction.png'))

FileNotFoundError: [Errno 2] No such file or directory: '../SAM_predictions_BBoxPoint/train/benign'

In [203]:
benign_predictions_dir=os.path.join('..', folder_name, 'valid' ,'benign')
benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
csv_file_name=os.path.join('..',folder_name,'valid','benign_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_BBoxPoint/valid/benign_metrics.csv
Average Dice Score: 0.33345001387875006
Average IOU: 0.2667893796437881
Average Pixel Wise Accuracy: 0.9265944698641393


In [204]:
benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
csv_file_name=os.path.join('..',folder_name,'train','benign_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score) 
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_BBoxPoint/train/benign_metrics.csv
Average Dice Score: 0.41365453454037593
Average IOU: 0.32805307887318086
Average Pixel Wise Accuracy: 0.9226797828801447


In [30]:
malignant_predictions_dir=os.path.join('..', folder_name, 'valid' ,'malignant')
malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
csv_file_name=os.path.join('..',folder_name,'valid','malignant_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

malignant (106)_mask_prediction.png
malignant (106)_mask_prediction.png
malignant (104)_mask_prediction.png
malignant (104)_mask_prediction.png
malignant (121)_mask_prediction.png
malignant (121)_mask_prediction.png
malignant (199)_mask_prediction.png
malignant (199)_mask_prediction.png
malignant (49)_mask_prediction.png
malignant (49)_mask_prediction.png
malignant (28)_mask_prediction.png
malignant (28)_mask_prediction.png
malignant (184)_mask_prediction.png
malignant (184)_mask_prediction.png
malignant (107)_mask_prediction.png
malignant (107)_mask_prediction.png
malignant (185)_mask_prediction.png
malignant (185)_mask_prediction.png
malignant (168)_mask_prediction.png
malignant (168)_mask_prediction.png
malignant (136)_mask_prediction.png
malignant (136)_mask_prediction.png
malignant (175)_mask_prediction.png
malignant (175)_mask_prediction.png
malignant (61)_mask_prediction.png
malignant (61)_mask_prediction.png
malignant (171)_mask_prediction.png
malignant (171)_mask_prediction.pn

In [206]:
malignant_predictions_dir=os.path.join('..', folder_name, 'train' ,'malignant')
malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
csv_file_name=os.path.join('..',folder_name,'train','malignant_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_BBoxPoint/train/malignant_metrics.csv
Average Dice Score: 0.40391546610342977
Average IOU: 0.2887636264066313
Average Pixel Wise Accuracy: 0.8743377422516299


### Point 

In [26]:
folder_name='SAM_predictions_Point'

#delete benign (195)_mask_2_mask.png image from train/benign folder

benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)
print(list_of_benign_predictions_dir)
#delete benign (195)_mask_2_mask.png image from valid/benign folder
if 'benign (195)_mask_2_mask_prediction.png' in list_of_benign_predictions_dir:
    os.remove(os.path.join(benign_predictions_dir,'benign (195)_mask_2_mask_prediction.png'))

FileNotFoundError: [Errno 2] No such file or directory: '../SAM_predictions_Point/train/benign'

In [27]:
benign_predictions_dir=os.path.join('..', folder_name, 'valid' ,'benign')
benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
csv_file_name=os.path.join('..',folder_name,'valid','benign_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,csv_file_name)
print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

benign (424)_mask_prediction.png
benign (424)_mask_prediction.png
benign (423)_mask_prediction.png
benign (423)_mask_prediction.png
benign (367)_mask_prediction.png
benign (367)_mask_prediction.png
benign (348)_mask_prediction.png
benign (348)_mask_prediction.png
benign (17)_mask_prediction.png
benign (17)_mask_prediction.png
benign (306)_mask_prediction.png
benign (306)_mask_prediction.png
benign (247)_mask_prediction.png
benign (247)_mask_prediction.png
benign (177)_mask_prediction.png
benign (177)_mask_prediction.png
benign (7)_mask_prediction.png
benign (7)_mask_prediction.png
benign (257)_mask_prediction.png
benign (257)_mask_prediction.png
benign (120)_mask_prediction.png
benign (120)_mask_prediction.png
benign (143)_mask_prediction.png
benign (143)_mask_prediction.png
benign (127)_mask_prediction.png
benign (127)_mask_prediction.png
benign (297)_mask_prediction.png
benign (297)_mask_prediction.png
benign (415)_mask_prediction.png
benign (415)_mask_prediction.png
benign (245)_mas

In [209]:
benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
csv_file_name=os.path.join('..',folder_name,'train','benign_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score) 
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_Point/train/benign_metrics.csv
Average Dice Score: 0.38446904752304195
Average IOU: 0.3297727250135217
Average Pixel Wise Accuracy: 0.6556376063408329


In [28]:
malignant_predictions_dir=os.path.join('..', folder_name, 'valid' ,'malignant')
malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
csv_file_name=os.path.join('..',folder_name,'valid','malignant_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

malignant (106)_mask_prediction.png
malignant (106)_mask_prediction.png
malignant (104)_mask_prediction.png
malignant (104)_mask_prediction.png
malignant (121)_mask_prediction.png
malignant (121)_mask_prediction.png
malignant (199)_mask_prediction.png
malignant (199)_mask_prediction.png
malignant (49)_mask_prediction.png
malignant (49)_mask_prediction.png
malignant (28)_mask_prediction.png
malignant (28)_mask_prediction.png
malignant (184)_mask_prediction.png
malignant (184)_mask_prediction.png
malignant (107)_mask_prediction.png
malignant (107)_mask_prediction.png
malignant (185)_mask_prediction.png
malignant (185)_mask_prediction.png
malignant (168)_mask_prediction.png
malignant (168)_mask_prediction.png
malignant (136)_mask_prediction.png
malignant (136)_mask_prediction.png
malignant (175)_mask_prediction.png
malignant (175)_mask_prediction.png
malignant (61)_mask_prediction.png
malignant (61)_mask_prediction.png
malignant (171)_mask_prediction.png
malignant (171)_mask_prediction.pn

In [211]:
malignant_predictions_dir=os.path.join('..', folder_name, 'train' ,'malignant')
malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
csv_file_name=os.path.join('..',folder_name,'train','malignant_metrics.csv')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,csv_file_name)

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

results_df.loc[len(results_df)]=[csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]

Metrics saved to  ../SAM_predictions_Point/train/malignant_metrics.csv
Average Dice Score: 0.4765595893259337
Average IOU: 0.3719763955847756
Average Pixel Wise Accuracy: 0.7316462829831193


In [212]:
print(results_df)

                                              Dataset       IOU  Dice Score  \
0    ../SAM_predictions_BBox/valid/benign_metrics.csv  0.302003    0.370326   
1    ../SAM_predictions_BBox/train/benign_metrics.csv  0.324584    0.407655   
2   ../SAM_predictions_BBox/valid/malignant_metric...  0.230592    0.321162   
3   ../SAM_predictions_BBox/train/malignant_metric...  0.281230    0.392978   
4   ../SAM_predictions_BBoxPoint/valid/benign_metr...  0.266789    0.333450   
5   ../SAM_predictions_BBoxPoint/train/benign_metr...  0.328053    0.413655   
6   ../SAM_predictions_BBoxPoint/valid/malignant_m...  0.240449    0.334292   
7   ../SAM_predictions_BBoxPoint/train/malignant_m...  0.288764    0.403915   
8   ../SAM_predictions_Point/valid/benign_metrics.csv  0.385636    0.439137   
9   ../SAM_predictions_Point/train/benign_metrics.csv  0.329773    0.384469   
10  ../SAM_predictions_Point/valid/malignant_metri...  0.271114    0.349678   
11  ../SAM_predictions_Point/train/malignant_metri..